In [142]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

def parse_fasta(fasta_filepath):
    id_to_seq = {}
    for record in SeqIO.parse(fasta_filepath, "fasta"):
        id_to_seq[record.id] = str(record.seq)
    return id_to_seq

def write_fasta(sequences, output_file, line_width=80):
    with open(output_file, "w+") as f:
        for header, seq in sequences.items():
            f.write(f">{header}\n")
            # Wrap sequence to line_width
            for i in range(0, len(seq), line_width):
                f.write(seq[i:i+line_width] + "\n")

def align(seq1, seq2):
    # seq1 = seq1.replace('-', 'X')
    # seq2 = seq2.replace('-', 'X')
    return pairwise2.align.globalxx(seq1, seq2)

def map_index(i, aligned_seq1, aligned_seq2):
    """Return the aligned character in seq2 corresponding to seq1[i]."""
    orig_index = 0
    for pos, char in enumerate(aligned_seq1):
        if char != "-":  # skip gaps
            orig_index += 1
        if orig_index == i:
            return aligned_seq1[pos], aligned_seq2[pos]
    return None  # if i is out of range

def check_mutation_sites(seq1, seq2, mutations, verbose=False):
    alignments = align(seq1, seq2)
    # aln = alignments[0]
    for aln in alignments:
        if verbose: print(format_alignment(*aln))
        aligned_seq1, aligned_seq2, score, begin, end = aln
        for mut in mutations:
            exp_wt_aa = mut[0]
            exp_mt_aa = mut[-1]
            idx = int(mut[1:-1])
            wt_aa, mt_aa = map_index(idx, aligned_seq1, aligned_seq2)
            if verbose: print(wt_aa, mt_aa)
            if exp_wt_aa != wt_aa:
                print(f'ERROR! Expected and found WT do not match: {exp_wt_aa} != {wt_aa}')
            if exp_mt_aa == mt_aa:
                print(f'GOTCHA! Expected and found MT match!')
    if verbose: print()

In [143]:
antigens_seqs = parse_fasta("antigens.fasta")

pdb_to_mutations = {
    '1OAN_A': ['A259W', 'T262R', 'S29K', 'T33V', 'A35M', 'G106D', 'T280P', 'F279W'],
    '7VDF_A': ['H355W', 'K380I', 'E432I'],
    '5WB0_A': ['A159L', 'V203I', 'L130D', 'V430Q', 'V449D', 'V112R', 'D209E', 'V231I', 'E453P', 'E80D', 'V155P'],
    '4jhw_A': ['S190F', 'V207L', 'D486H', 'E487Q', 'F488W', 'D489H', 'S215P'],
    '6VSB_A': ['F817P', 'A892P', 'A899P', 'A942P'],
}

In [144]:
## test against casp12

casp12_seqs = parse_fasta("casp12_no_duplicates.fasta")
blast_results_df = pd.read_csv('antigens_vs_casp12.tsv', sep='\t', names='qseqid sseqid pident length qlen slen evalue bitscore'.split())

for i, row in blast_results_df.iterrows():
    pdb_query = row['qseqid']
    pdb_in_db = row['sseqid']

    # get mutations, skip if pdb_chain not relevant
    if pdb_query not in pdb_to_mutations:
        continue
    mutations = pdb_to_mutations[pdb_query]

    # get sequences
    seq_query = antigens_seqs[pdb_query]
    seq_in_db = casp12_seqs[pdb_in_db]
    
    # run the check!
    check_mutation_sites(seq_query, seq_in_db, mutations, verbose=False)

In [145]:
## test against cdna117k

casp12_seqs = parse_fasta("cdna117k_plus_t2837.fasta")
blast_results_df = pd.read_csv('antigens_vs_cdna.tsv', sep='\t', names='qseqid sseqid pident length qlen slen evalue bitscore'.split())

for i, row in blast_results_df.iterrows():
    pdb_query = row['qseqid']
    pdb_in_db = row['sseqid']

    # get mutations, skip if pdb_chain not relevant
    if pdb_query not in pdb_to_mutations:
        continue
    mutations = pdb_to_mutations[pdb_query]

    # get sequences
    seq_query = antigens_seqs[pdb_query]
    seq_in_db = casp12_seqs[pdb_in_db]
    
    # run the check!
    check_mutation_sites(seq_query, seq_in_db, mutations, verbose=False)

In [146]:
## test against megascale

casp12_seqs = parse_fasta("megascale_train_test.fasta")
blast_results_df = pd.read_csv('antigens_vs_megascale.tsv', sep='\t', names='qseqid sseqid pident length qlen slen evalue bitscore'.split())

for i, row in blast_results_df.iterrows():
    pdb_query = row['qseqid']
    pdb_in_db = row['sseqid']

    # get mutations, skip if pdb_chain not relevant
    if pdb_query not in pdb_to_mutations:
        continue
    mutations = pdb_to_mutations[pdb_query]

    # get sequences
    seq_query = antigens_seqs[pdb_query]
    seq_in_db = casp12_seqs[pdb_in_db]
    
    # run the check!
    check_mutation_sites(seq_query, seq_in_db, mutations, verbose=False)

In [149]:
## test against proteinmpnn training data

casp12_seqs = parse_fasta("proteinmpnn.fasta")
blast_results_df = pd.read_csv('antigens_vs_proteinmpnn.tsv', sep='\t', names='qseqid sseqid pident length qlen slen evalue bitscore'.split())

for i, row in blast_results_df.iterrows():
    pdb_query = row['qseqid']
    pdb_in_db = row['sseqid']

    # get mutations, skip if pdb_chain not relevant
    if pdb_query not in pdb_to_mutations:
        continue
    mutations = pdb_to_mutations[pdb_query]

    # get sequences
    seq_query = antigens_seqs[pdb_query]
    seq_in_db = casp12_seqs[pdb_in_db]
    
    # run the check!
    check_mutation_sites(seq_query, seq_in_db, mutations, verbose=False)

In [141]:
for filename in ['casp12_no_duplicates.fasta', 'cdna117k_plus_t2837.fasta', 'megascale_train_test.fasta', 'antigens.fasta']:
    seqs = parse_fasta(filename)
    fixed_seqs = {header: seq.replace('-', 'X') for header, seq in seqs.items()}
    write_fasta(fixed_seqs, filename)